<a href="https://colab.research.google.com/github/marcomag416/MLDL/blob/main/bisenet_2b_colab_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminary code
Feel free to delete/skip this part if run in a persistent environment

In [1]:
#download bisenet model from official repository
import sys
import requests
from zipfile import ZipFile
from io import BytesIO
model_url = "https://github.com/ooooverflow/BiSeNet/archive/refs/heads/master.zip"

# Send a GET request to the URL
response = requests.get(model_url)
# Check if the request was successful
if response.status_code == 200:
    #print(response.content)
    # Open the downloaded bytes and extract them
    with ZipFile(BytesIO(response.content)) as zip_file:
        zip_file.extractall('./')
    print('Download and extraction complete!')

sys.path.insert(0, './BiSeNet-master')

Download and extraction complete!


In [1]:
#NOTE: run this cell to download and extract cityscape dataset

# Define the path to the dataset
dataset_path = 'https://drive.usercontent.google.com/download?id=1Qb4UrNsjvlU-wEsR9d7rckB0YS_LXgb2&export=download&authuser=0&confirm=t&uuid=9b831c4d-351d-4a8f-b7e4-213114a9e2e0&at=APZUnTVYj5OMDKe3JasX4A6A0iTJ:1716458171729'  # Replace with the path to your dataset

# Send a GET request to the URL
response = requests.get(dataset_path)
# Check if the request was successful
if response.status_code == 200:
    #print(response.content)
    # Open the downloaded bytes and extract them
    with ZipFile(BytesIO(response.content)) as zip_file:
        zip_file.extractall('./dataset')
    print('Download and extraction complete!')

dataset_path = "./dataset/Cityscapes/Cityspaces"

b'<!DOCTYPE html><html><head><title>Google Drive - Quota exceeded</title><meta http-equiv="content-type" content="text/html; charset=utf-8"/><style nonce="7ZD-y-0kOSrVvYBIhcjoPw">.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial,sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-align:left}.uc-error-caption,.uc-warning-caption{color:#222;font-size:16px}#uc-download-link{text-decoration:none}.uc-name-size a{color:#15c;text-decoration:none}.uc-name-size a:visited{color:#61c;text-decoration:none}.uc-name-size a:active{color:#d14836;text-decoration:none}.uc-footer{color:#777;font-size:11px;padding-bottom:5ex;padding-top:5ex;text-align:center}.uc-footer a{c

BadZipFile: File is not a zip file

In [2]:
#if the above doesn't work exctract zip file directly from google drive
from google.colab import drive
drive.mount('/content/drive')

with ZipFile("/content/drive/MyDrive/Colab Notebooks/dataset/Cityscapes.zip", 'r') as zip_ref:
    zip_ref.extractall("./dataset")

dataset_path = "./dataset/Cityscapes/Cityspaces"

Mounted at /content/drive


In [3]:
#install and import wandb for data collecting
!pip install wandb
import wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.9/288.9 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00


# 2b BiseNet training and validation

In [4]:
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T
from torch.utils.data import DataLoader
import os
import albumentations as A
from albumentations.pytorch import ToTensorV2

if __name__ != '__main__':
    raise Exception("This script should not be imported; it should be run directly.")

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

class Cityscapes(Dataset):
    def __init__(self, root_dir, split, transforms=None, label_type='gtFine_labelTrainIds'):
        self.root_dir = root_dir
        self.split = split
        self.transforms = transforms
        self.label_type = label_type

        self.images_dir = f"{root_dir}/images/{split}"
        self.labels_dir = f"{root_dir}/gtFine/{split}"

        self.image_paths = []
        self.label_paths = []

        # Manually iterate over directories
        cities = [city for city in os.listdir(self.images_dir) if os.path.isdir(f"{self.images_dir}/{city}")]
        for city in cities:
            img_dir_city = f"{self.images_dir}/{city}"
            lbl_dir_city = f"{self.labels_dir}/{city}"

            if not os.path.isdir(img_dir_city) or not os.path.isdir(lbl_dir_city):
                continue

            for img_file in os.listdir(img_dir_city):
                if img_file.endswith('_leftImg8bit.png'):
                    img_path = f"{img_dir_city}/{img_file}"
                    lbl_file = img_file.replace('_leftImg8bit.png', f'_{self.label_type}.png')
                    lbl_path = f"{lbl_dir_city}/{lbl_file}"

                    if os.path.isfile(img_path) and os.path.isfile(lbl_path):
                        self.image_paths.append(img_path)
                        self.label_paths.append(lbl_path)
                    else:
                        print(f"Warning: Image or label file not found for {img_file}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        lbl_path = self.label_paths[idx]

        image = Image.open(img_path).convert('RGB')
        label = Image.open(lbl_path)

        image = np.array(image)
        label = np.array(label)

        if self.transforms:
            augmented = self.transforms(image=image, mask=label)
            image, label = augmented['image'], augmented['mask']

        return image, label


# Example usage
image_transforms = A.Compose([
    A.Resize(512, 1024),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

dataset = Cityscapes(root_dir=dataset_path, split='train', transforms=image_transforms)   #replace cityscapes dataset path here!!
val_dataset = Cityscapes(root_dir=dataset_path, split='val', transforms=image_transforms)

print(f"Dataset size: {len(dataset)}")
print(f"Val_Dataset size: {len(val_dataset)}")
"""
for idx, (im, lb) in enumerate(train_dataloader):
    print(f"Batch {idx + 1}: Image batch shape: {im.shape}, Label batch shape: {lb.shape}")

    unique_values, counts = np.unique(lb.numpy(), return_counts=True)
    print("Unique values in the label tensor:", unique_values)
    print("Counts of unique values:", counts)

    if idx >= 4:  # Print information for first 5 batches
        break
"""

Dataset size: 1572
Val_Dataset size: 500


'\nfor idx, (im, lb) in enumerate(train_dataloader):\n    print(f"Batch {idx + 1}: Image batch shape: {im.shape}, Label batch shape: {lb.shape}")\n\n    unique_values, counts = np.unique(lb.numpy(), return_counts=True)\n    print("Unique values in the label tensor:", unique_values)\n    print("Counts of unique values:", counts)\n\n    if idx >= 4:  # Print information for first 5 batches\n        break\n'

In [5]:

def train(model, optimizer_train, dataloader, loss_fn_train):
    model.train()  # Set the model to training mode
    train_loss = 0.0
    total = 0

    for idx, (inputs_train, targets_train) in enumerate(dataloader):
        inputs_train = inputs_train.to(device)
        targets_train = targets_train.to(device, dtype=torch.long)  # Move data to the appropriate device

        optimizer_train.zero_grad()  # Zero out gradients from the previous iteration
        outputs_train, _, _ = model(inputs_train)  # Forward pass
        # print( "train")
        loss = loss_fn_train(outputs_train, targets_train)  # Calculate the loss

        loss.backward()  # Backward pass
        optimizer_train.step()  # Update the weights

        wandb.log({"train/Batch loss": loss})

        train_loss += loss.item() * inputs_train.size(0)  # Accumulate the total loss
        _, predicted_train = outputs_train.max(1)
        total += targets_train.size(0)

    # Calculate average loss for the epoch
    avg_loss = train_loss / total

    wandb.log({"train/Epoch loss": avg_loss})

    return avg_loss


def compute_iou(pred, target, num_classes):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)

    for cls in range(num_classes):
        pred_inds = (pred == cls)
        target_inds = (target == cls)
        intersection = (pred_inds[target_inds]).sum().item()
        union = pred_inds.sum().item() + target_inds.sum().item() - intersection
        if union == 0:
            ious.append(float('nan'))  # If there is no union, set IoU to NaN
        else:
            ious.append(intersection / union)

    return np.array(ious)

def eval(model, dataloader, loss_fn, device, num_classes=19):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    total = 0
    all_ious = []  # List to store IoUs for each batch

    with torch.no_grad():  # Disable gradient calculation during inference
        for inputs_test, targets_test in dataloader:
            inputs_test, targets_test = inputs_test.to(device), targets_test.to(device, dtype=torch.long)

            outputs_test = model(inputs_test)  # Forward pass
            loss = loss_fn(outputs_test, targets_test)  # Calculate the loss

            test_loss += loss.item() * inputs_test.size(0)  # Accumulate the total loss
            _, predicted_test = outputs_test.max(1)
            total += targets_test.size(0)

            # Compute IoU for this batch
            batch_ious = compute_iou(predicted_test, targets_test, num_classes)
            all_ious.append(batch_ious)

    # Calculate average loss
    avg_loss = test_loss / total

    wandb.log({})

    # Calculate mean IoU
    all_ious = np.array(all_ious)
    mean_iou = np.nanmean(all_ious, axis=0)  # Mean IoU for each class
    miou = np.nanmean(mean_iou)  # Mean IoU across all classes

    wandb.log({"val/Validation loss": avg_loss, "val/mIoU": miou})

    return avg_loss, miou

In [6]:
from model.build_BiSeNet import BiSeNet
from torch import nn
from torch.optim.lr_scheduler import PolynomialLR


# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

context_path = 'resnet18'

#save hyperparameters
wandb.init(project="BiSeNet")
config = wandb.config
config.learning_rate = 2.5e-2
config.max_epochs = 50
config.batch_size = 16
config.weight_decay = 1e-4
config.dataset = "Cityscapes"
config.scheduler = "Poloynomial"
config.polyPower = 0.9

# create dataloaders
train_dataloader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

# Initialize the model
model = BiSeNet(num_classes=19, context_path=context_path).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
scheduler = PolynomialLR(optimizer, total_iters=config.max_epochs, power=config.polyPower)

# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 113MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 153MB/s]


In [ ]:
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
for epoch in range(config.max_epochs):
    wandb.log({"Epoch": epoch+1, "train/Learning rate": scheduler.get_last_lr()})
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou = eval(model, val_dataloader, loss_fn, device=device)
    scheduler.step()
    print(f"Loss: {avg_loss}, mIoU: {miou*100:.2f}%")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")
wandb.finish()

Loss: 1.7000539512634278, mIoU: 7.45%
Loss: 1.2350229291915893, mIoU: 11.97%
Loss: 1.037554874420166, mIoU: 14.99%
Loss: 1.0268001165390015, mIoU: 15.19%
Loss: 1.0236913499832154, mIoU: 15.07%
Loss: 0.8350193090438843, mIoU: 17.55%
Loss: 1.5417133445739746, mIoU: 13.44%
Loss: 0.9187056980133057, mIoU: 17.84%
Loss: 1.0566452655792236, mIoU: 17.04%
Loss: 0.9970059309005738, mIoU: 17.99%
Loss: 0.5745296468734741, mIoU: 23.18%
Loss: 0.8567861166000367, mIoU: 18.66%
Loss: 0.8493251929283142, mIoU: 17.77%
Loss: 0.8009709167480469, mIoU: 19.46%
Loss: 1.125888858795166, mIoU: 15.10%
Loss: 0.6520994501113891, mIoU: 22.48%
Loss: 0.579279592514038, mIoU: 22.05%
Loss: 0.6537197360992432, mIoU: 23.28%
Loss: 0.6646755537986755, mIoU: 23.01%
Loss: 1.4651100149154663, mIoU: 13.49%
Loss: 0.6270851788520813, mIoU: 21.36%
Loss: 0.8198208003044128, mIoU: 17.72%
Loss: 0.8203187499046326, mIoU: 22.28%
Loss: 0.5532145853042603, mIoU: 24.81%
Loss: 1.0271180362701415, mIoU: 16.90%
Loss: 0.6107429151535034, mIo